In this project, we are simulating the molecular dynamics of a number of argon atoms. The first thing we do is assign values to all relevant parameters, and initialise the matrices which we are going to use in this project. We have used natural units for non assigned variables (mass, energy etc)

In [4]:
# Import everything

import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assign values to relevent paramters

# gas phase, Td = 3, rho = 0.3
# solid phase, Td = 0.5, rho = 1.2
# liquid phase, Td = 1, rho = 0.8
Td = 1    # Desired temperature
rho = 0.8    # Density of argon
Nuc = 3    # Number of unit cells
N = 4*Nuc**3    # Number of particles
L = (N/rho)**(1/3)    # Total lenght of the box
mu, sigma = 0, 3  
simulationTime = 20    # Total simulation time
h = 0.004    # Time between two simulation steps
steps = int(simulationTime/h)    # Number of simulation steps
t = 0    # Initial time
rescalingsteps = int(steps/2)    # Number of times the velocity is rescaled

# Initialise matrices

dist = np.zeros((N,N))  
xdif = np.zeros((N,N))
ydif = np.zeros((N,N))
zdif = np.zeros((N,N))
v = np.zeros((N,3))
v_wiggle = np.zeros((N,3))
allFLJ = np.zeros((3,N,N))
F = np.zeros((N,3))
location = np.zeros((N,3))
time = np.linspace(1, steps, steps)


The atoms are placed in an FCC lattice, which is the minimum energy configuration for the Lennard-Jones system. We also assign random velocities, following a Gaussian distribution, in all three directions to all particles. Because these velocities are randomly distributed around zero, the center of mass velocity is not exactly zero. We therefor have to correct the velocities with the center of mass velocity to prevent the system from drifting in space.

In [ ]:
def initial_location():
    loc = []
    for x in range(Nuc):
        for y in range(Nuc):
            for z in range(Nuc):
                loc.append([x, y, z])
                loc.append([x, 0.5 + y, 0.5 + z])
                loc.append([0.5 + x, y, 0.5 + z])
                loc.append([0.5 + x, 0.5 + y, z])
    return np.array(loc) * L/Nuc


def initial_velocity(mu, sigma):
    v = np.random.normal(mu, sigma, size=(N,3))
    v = v - np.average(v, axis=0)
    return v

After we've placed the atoms at their initial position and assigned the initial velocities, we can calculate the force due to the Lennard-Jones potential. The minimum image convention is used to ensure that the interaction is with the nearest copy of each of the particles.

In [ ]:
def calculate_forces(location):
    for i in range(0,N):
        for j in range(0,N):
            if j == i:
                continue
            
            r = location[i,:]-location[j,:]
            r = r - np.rint(r/L)*L
            dist[i,j] = math.sqrt(np.sum(r**2))
            xdif = r[0]
            ydif = r[1]
            zdif = r[2]          
        
            allFLJ[0,i,j] = xdif*(48*dist[i,j]**-14 - 24*dist[i,j]**-8)
            allFLJ[1,i,j] = ydif*(48*dist[i,j]**-14 - 24*dist[i,j]**-8)
            allFLJ[2,i,j] = zdif*(48*dist[i,j]**-14 - 24*dist[i,j]**-8)        
    
        F[i,0] = np.sum(allFLJ[0,i,:])
        F[i,1] = np.sum(allFLJ[1,i,:])
        F[i,2] = np.sum(allFLJ[2,i,:])
    
    return F, dist

The integration of the equation of motion is carried out using the velocity-Verlet algorithm. We impose periodic boundary conditions to ensure that the particles will stay in the box. In this step, we also calculate the kinetic and potential energy, and make an histogram of the inner distances which will be used to derive the pair correlation function.

In [ ]:
def update_quantities(v, F, location):
    v_wiggle = v + h*F/2
    location = np.mod(location + h*v_wiggle ,L)
    F, dist = calculate_forces(location)
    v = v_wiggle + h*F/2
    K = 0.5*np.sum(v**2)
    newdist = dist[dist != 0]
    Pot = 0.5*np.sum(4*(newdist**-12 - newdist**-6))
    hist = np.histogram(newdist, bins=50, range=(0.1*L/(2*50),L/2))[0]
    virial = 0.5*np.sum((48*newdist**-13 - 24*newdist**-7)*newdist)
    return v, F, location, K, Pot, hist, virial
 

To arrive at the desired temperature, we rescale the velocity in each step untill we say the system is in equilibrium.

In [5]:
def rescale_velocity(v, K):
    lmbda = np.sqrt((N-1)*3*Td/(2*K))
    v = v*lmbda
    return v 
     

Now we can perform the simulation. We plot the kinetic, potential and total energy of the system over time. It is clearly visible that when we stop rescaling the velocity, the total energy is indeed conserved. The specific heat is calculated from Lebowitz' formula, and the pressure is calculated using the virial theorem. We use the inner distance histogram to derive the pair correlation function, which is then plotted as a function over the inner distances.
 

In [7]:
def perform_simulation():
    
    Ek = []
    Ep = []
    histogram = []
    Pvirial = []
    
    location = np.asarray(initial_location())
    v = initial_velocity(mu, sigma)
    F = calculate_forces(location)[0]    

    for t in range (1,(steps+1)):
        
        if t >= 1:
            v, F, location, K, Pot, hist, virial = update_quantities(v, F, location)
            Ek.append(K)
            Ep.append(Pot)
            histogram.append(hist)
            Pvirial.append(virial)
            
        if t in range (1,rescalingsteps):
            v = rescale_velocity(v, K)
        
    Ek = np.array(Ek)
    Ep = np.array(Ep)
    Et = Ek + Ep
    plt.plot(time, Ek)
    plt.plot(time, Ep)
    plt.plot(time, Et)
    plt.legend(("Potential energy", "Kinetic energy", "Total energy"))
    plt.title("Energies of %g argon atoms"%(N))
    plt.xlabel(("t"))
    plt.ylabel(("E"))
    plt.show()
    
    T = np.average(2*Ek[rescalingsteps:]/(3*N))
    print("Average temperature is " + str(T))
    finalT = 2*K/(3*N)
    print("Final temperature is " + str(finalT))
   
    n = np.average(np.array(histogram), axis=0)
    Range = np.linspace(0.1*L/(2*50), L/2, 50)
    g = 2/(rho*(N-1)) * n/(4*math.pi*Range**2*(Range[1]-Range[0]))
    plt.plot(Range, g)
    if Td == 1 and rho == 0.8:
        plt.legend(("Liquid"))
    if Td == 3 and rho == 0.3:
        plt.legend(("Gas"))
    if Td == 0.5 and rho == 1.2:
        plt.legend(("Solid"))
    plt.title(("Pair correlation function"))
    plt.xlabel(("r"))
    plt.ylabel(("g"))
    plt.show()
    
    cV = 3*np.average(Ek[rescalingsteps:])**2/(2*np.average(Ek[rescalingsteps:])**2-3*N*np.var(Ek[rescalingsteps:]))
    print("Specific heat is " +str(cV))
    
    Pvirial = np.array(Pvirial)
    P = 1 + 1/(3*N*T)*np.average(Pvirial[rescalingsteps:])
    print("Pressure is " +str(P))
    
    #return Ek, Ep, Et, T, g, finalT, cV, P,

perform_simulation()


Average temperature is 0.996546515721
Final temperature is 0.951461361449
Specific heat is 2.84601239233
Pressure is 1.74161283597
